In [3]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import arviz as az
import pymc as pm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cellbayesassay as cba
import attila_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%connect_info

{
  "shell_port": 54675,
  "iopub_port": 54676,
  "stdin_port": 54677,
  "control_port": 54679,
  "hb_port": 54678,
  "ip": "127.0.0.1",
  "key": "101b67a5-7aa467737c25585b32a7c69c",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3",
  "jupyter_session": "/Users/jonesa7/CTNS/notebooks/2024-02-14-cell-bayes/2024-02-14-cell-bayes.ipynb"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-d90e05fd-a696-4064-9ea9-5999e6f10f0b.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
